# Morphology: Binary Methods

In [ ]:
%matplotlib inline

import time

import numpy as np

import matplotlib.image as img
import matplotlib.pyplot as plt

from skimage import data, io
from skimage.util import img_as_ubyte, invert

from skimage.morphology import disk, square
from skimage.morphology import thin, medial_axis
from skimage.morphology import area_opening
from skimage.morphology import convex_hull_image, convex_hull_object

from scipy.ndimage import binary_dilation, binary_erosion
from scipy.ndimage import binary_opening, binary_closing

In [ ]:
def show_images(I, titles=None):
    fig, ax = plt.subplots(1, len(I), figsize=(12,12))
    
    for i in np.arange(0,len(I)):
        ax[i].imshow(I[i], cmap='gray')
        ax[i].set_axis_off()
        if titles != None:
            ax[i].set_title(titles[i])

In [ ]:
I = io.imread("../../images/blobs.png", as_gray=True)
I = img_as_ubyte(I)

B1 = np.where(I<70, False, True)
B2 = np.where(I<128, False, True)
B3 = np.where(I<150, False, True)

show_images([B1,B2,B3], ['Low Threshold','Medium Threshold','High Threshold'])

In [ ]:
for N in [1,2]:
    print(disk(N),'\n')
    
for N in [3,5]:
    print(square(N),'\n')

In [ ]:
B = np.copy(B2)

SE = disk(1)
OP = binary_dilation # dilation, erosion, opening, closing

Bk = [B]
Tk = [0]

for k in [1,2]:
    BSE = OP(B, SE, iterations=k)
    Bk.append(BSE)
    Tk.append(k)

show_images(Bk, Tk)

In [ ]:
B = np.copy(B2)

SE4 = disk(1)
SE8 = square(3)

B4 = np.bitwise_and(B, np.invert(binary_erosion(B, SE4))) # Boundary w/8-connectivity
B8 = np.bitwise_and(B, np.invert(binary_erosion(B, SE8))) # Boundary w/4-connectivity
SD = B4^B8

show_images([B4,B8,SD],['SE4','SE8','Set Difference'])

In [ ]:
B = np.copy(B2)

M1 = np.bitwise_and(B, invert(thin(B)))
M2 = np.bitwise_and(B, invert(medial_axis(B)))
M,D = medial_axis(B, return_distance=True)

show_images([M1,M2,D],['Thinning','Medial Axis','Distance Map'])

In [ ]:
B = area_opening(B2, 500, connectivity=1)

C1 = convex_hull_image(B)
C2 = convex_hull_object(B, connectivity=1)

C1 = img_as_ubyte(C1)//2
C2 = img_as_ubyte(C2)//2

C1[B] = 255
C2[B] = 255

show_images([B,C1,C2],['Area Opening','Convex Hull Image','Convex Hull Object'])